In [1]:
import os
import datetime
import copy
import re
import yaml
import uuid
import warnings
import time
import inspect

import numpy as np
import pandas as pd
from functools import partial, reduce
from random import shuffle
import random

import torch
from torch import nn, optim
from torch import nn
from torch.nn import functional as F
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader
from torchvision.models import resnet
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torchvision.models.resnet import ResNet, BasicBlock
from torchvision.datasets import MNIST
from tqdm.autonotebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn import metrics as mtx
from sklearn import model_selection as ms
import os
import shutil
import warnings
import torch
import torch.nn as nn
import torchvision.models as models
from sklearn.preprocessing import StandardScaler
import numpy as np

warnings.filterwarnings("ignore")

/tmp/ipykernel_320446/1343771631.py:28: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [41]:
# This function is to flatten images
source_folder = "/home/lujun/local/DLMI-Classification/data/raw/testset/"
destination_folder = "/home/lujun/local/DLMI-Classification/data/raw/testset_flattened/"
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)
    for person_folder in os.listdir(source_folder):
        person_folder_path = os.path.join(source_folder, person_folder)
        if os.path.isdir(person_folder_path):
            for file in os.listdir(person_folder_path):
                file_path = os.path.join(person_folder_path, file)
                if os.path.isfile(file_path):
                    target_file_name = f"{person_folder}_{file}"
                    target_file_path = os.path.join(
                        destination_folder, target_file_name
                    )
                    shutil.copy(file_path, target_file_path)
print("Flattened Images has benn done", destination_folder)

Flattened Images has benn done /home/lujun/local/DLMI-Classification/data/raw/testset_flattened/


In [45]:
clinical_annotation_dataframe = pd.read_csv(
    "/home/lujun/local/DLMI-Classification/data/raw/clinical_annotation.csv"
)

clinical_post_processing = pd.DataFrame()

for index, row in clinical_annotation_dataframe.iterrows():
    target_folder = (
        "/home/lujun/local/DLMI-Classification/data/raw/testset/" + row["ID"] + "/"
    )
    if os.path.exists(target_folder):
        for file in os.listdir(target_folder):
            new_row = row.copy()
            file_path = os.path.join(target_folder, file)
            if os.path.isfile(file_path):
                new_row["image_id"] = f"{row.ID}_{file[:-4]}"
                clinical_post_processing = clinical_post_processing.append(
                    new_row, ignore_index=True
                )

clinical_post_processing.drop(columns=["Unnamed: 0"], inplace=True)
clinical_post_processing.to_csv("clinical_annotation_test.csv")

In [11]:
from PIL import Image
from datetime import datetime


def calculate_age(dob_str):
    formats = ["%m/%d/%Y", "%d-%m-%Y"]  # Two different format at the same time!
    for format_string in formats:
        try:
            dob_date = datetime.strptime(dob_str, format_string)
            current_date = datetime.now()
            age = (
                current_date.year
                - dob_date.year
                - (
                    (current_date.month, current_date.day)
                    < (dob_date.month, dob_date.day)
                )
            )
            return age
        except ValueError:
            continue


def encode_gender(gender):
    if gender == "F" or gender == "f":  # Two different gender value F and f ....
        return 0
    elif gender == "M" or gender == "m":
        return 1
    else:
        raise ValueError(
            "Invalid gender value. Expected 'F' or 'M', but received: {}".format(gender)
        )


class DLMICustomDataset(Dataset):
    def __init__(self, data, transform=None, flag="trainset_flattened"):
        self.data = data
        self.transform = transform
        self.flag = flag

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_ID = self.data.iloc[idx, 6]
        img_path = "data/post-processed/" + self.flag + "/" + str(img_ID) + ".jpg"
        # Assuming the image path is in the first column
        image = Image.open(img_path).convert("RGB")
        label = torch.tensor(
            int(self.data.iloc[idx, 2]), dtype=torch.float
        )  # Assuming label is in the second column
        if self.transform:
            image = self.transform(image)
        gender = torch.tensor(encode_gender(self.data.iloc[idx, 3]), dtype=torch.long)
        age = torch.tensor(calculate_age(self.data.iloc[idx, 4]), dtype=torch.float32)
        lymph_count = torch.tensor(self.data.iloc[idx, 5], dtype=torch.float32)
        clinical_data = torch.stack((gender, age, lymph_count))

        return image, clinical_data, label

In [12]:
class HybridModel(nn.Module):
    def __init__(self, num_classes, mlp_input_dim, mlp_hidden_dim):
        super(HybridModel, self).__init__()
        self.resnet18 = models.resnet18(pretrained=False)
        num_ftrs = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Linear(num_ftrs, num_ftrs)
        self.mlp = MLP(
            input_dim=mlp_input_dim, hidden_dim=mlp_hidden_dim, output_dim=num_classes
        )

        self.linear_layers = nn.ModuleList(
            [
                nn.Linear(num_ftrs if i == 0 else num_classes, num_classes)
                for i in range(4)
            ]
        )

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, image_data, clinical_data):
        image_output = self.resnet18(image_data)
        for layer in self.linear_layers:
            image_output = layer(image_output)
        mlp_output = self.mlp(clinical_data)
        image_output = self.sigmoid(image_output)
        mlp_output = self.sigmoid(mlp_output)
        final_output = (image_output + mlp_output) / 2
        return final_output


class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Training

overfittting problem

The model structure is good or not?

Data augmentation is avaialble or not?


learning rate is good or not? 

data is imbalanced 

Is the modle should be frozen

Initiallise the weight： 

In [16]:
import torchvision.transforms as transforms
from torchvision.datasets import DatasetFolder
from torch.utils.data import random_split

# High learning rate high accuracy but overfitting 0.1 too high
# 0.05 a good choice
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = pd.read_csv(
    "/home/lujun/local/DLMI-Classification/data/post-processed/clinical_annotation_train.csv"
)
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
custom_dataset = DLMICustomDataset(data=data, transform=transform)

train_size = int(0.8 * len(custom_dataset))
val_size = len(custom_dataset) - train_size
train_dataset, val_dataset = random_split(custom_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=12, shuffle=False)

model = HybridModel(num_classes=1, mlp_input_dim=3, mlp_hidden_dim=6).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

num_epochs = 100
best_loss = float("inf")
unchanged_count = 0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, clinical, labels in train_loader:
        inputs, clinical, labels = (
            inputs.to(device),
            clinical.to(device),
            labels.to(device),
        )
        optimizer.zero_grad()
        outputs = model(inputs, clinical)
        labels = labels.unsqueeze(1)
        labels.requires_grad = True
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    # Validation
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, clinical, labels in val_loader:
            inputs, clinical, labels = (
                inputs.to(device),
                clinical.to(device),
                labels.to(device),
            )
            outputs = model(inputs, clinical)
            labels = labels.unsqueeze(1)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            total += labels.size(0)
            correct += ((outputs - labels) < 0.5).sum().item()
    val_loss = val_loss / len(val_loader.dataset)  # Calculate validation loss2
    val_accuracy = correct / total  # Calculate validation accuracy
    print(
        f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}"
    )

    if epoch_loss < best_loss:
        best_loss = epoch_loss
        unchanged_count = 0
    else:
        unchanged_count += 1

    if unchanged_count >= 5:
        print("Loss has not changed for 5 consecutive epochs. Stopping training.")
        break

KeyboardInterrupt: 

In [ ]:
# Save the trained model
torch.save(model.state_dict(), "trained_model_add_sigmoid_0.01.pth")

In [30]:
# Evaluate the model on the test set and output predictions
model.eval()
all_predictions = []
test_data = pd.read_csv(
    "/home/lujun/local/DLMI-Classification/data/post-processed/clinical_annotation_test.csv"
)
test_dataset = DLMICustomDataset(
    data=test_data, transform=transform, flag="testset_flattened"
)

with torch.no_grad():
    all_predictions = []  # Initialize list to store predictions
    for i in range(len(test_dataset)):
        inputs, clinical, _ = test_dataset[i]  # Get inputs, clinical data, and label
        inputs, clinical = inputs.unsqueeze(0).to(device), clinical.unsqueeze(0).to(
            device
        )  # Add batch dimension and move to device
        outputs = model(inputs, clinical)  # Forward pass
        all_predictions.append(outputs.item())  # Append predicted class to list
        # Print example outputs for debugging
        if i == 0:
            print("Example combined outputs:", outputs)

# Output all predictions
print("All Predictions:", all_predictions)

Example combined outputs: tensor([[0.5000]], device='cuda:0')
All Predictions: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 

In [26]:
from collections import defaultdict

person_predictions = defaultdict(list)
images_names = []
source_folder_path = (
    "/home/lujun/local/DLMI-Classification/data/post-processed/testset_flattened"
)
for person_folder in os.listdir(source_folder_path):
    images_names.append(person_folder[:-4])

for filename, prediction in zip(images_names, all_predictions):
    person_number = filename.split("_")[0]  # Extract person number from filename
    person_predictions[person_number].append(prediction)

person_labels = {}
for person_number, predictions in person_predictions.items():
    count_0 = predictions.count(0)
    count_1 = predictions.count(1)
    print(f"This is the count: {count_0} + {count_1}")
    if count_0 > count_1:
        person_labels[person_number] = 0
    else:
        person_labels[person_number] = 1

# Output the final labels after majority voting
for person_number, label in person_labels.items():
    print(f"Person {person_number}: Predicted Label {label}")

TypeError: list.count() takes exactly one argument (0 given)

In [32]:
df = pd.DataFrame({"Image_Name": images_names, "Prediction": all_predictions})

In [42]:
df[["Person", "Number"]] = df["Image_Name"].str.split("_", 1, expand=True)

In [46]:
grouped_df = (
    df.groupby("Person")["Prediction"]
    .agg(["mean", "max", "min", "median"])
    .reset_index()
)

In [47]:
grouped_df

,Person,mean,max,min,median
0,P108,0.804301,1.0,0.5,1.000000
1,P114,0.826440,1.0,0.5,0.999999
2,P119,0.793394,1.0,0.5,1.000000
3,P120,0.799222,1.0,0.5,1.000000
4,P132,0.826870,1.0,0.5,1.000000
5,P133,0.851475,1.0,0.5,1.000000
6,P138,0.751155,1.0,0.5,0.779365
7,P139,0.841659,1.0,0.5,1.000000
8,P14,0.770479,1.0,0.5,0.956484
9,P143,0.826129,1.0,0.5,1.000000


In [93]:
test_dataset

Pretraining the resnet model

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import YourDataset  # 自定义数据集类，需根据实际情况修改

# 定义预处理转换
preprocess = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# 加载数据集
dataset = YourDataset(
    root="path/to/your/dataset", transform=preprocess
)  # 修改为你的数据集路径
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 加载预训练的ResNet模型
resnet = models.resnet18(pretrained=True)

# 冻结参数，只更新最后一层
for param in resnet.parameters():
    param.requires_grad = False

# 替换最后一层（全连接层）为自定义的嵌入层
num_features = resnet.fc.in_features
embedding_size = 128  # 嵌入向量的维度
resnet.fc = nn.Linear(num_features, embedding_size)

# 定义损失函数和优化器
criterion = nn.TripletMarginLoss()  # 使用三元组损失
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)

# 训练模型
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        # 正向传播
        embeddings = resnet(images)

        # 计算损失
        loss = criterion(embeddings, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(dataset)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# 保存模型
torch.save(resnet.state_dict(), "resnet18_embedding_model.pth")